# VN2 Submission: TimesFM 2.5 Quantile + Covariates Demo

**Purpose**: Demonstrate TimesFM 2.5's quantile head WITH COVARIATES for VN2 inventory planning

**Enhancement**: Combines `forecast_with_covariates()` + `use_continuous_quantile_head=True`

**Approach**:
1. Load all 599 SKUs from VN2
2. Prepare static (store, product group) + dynamic (month, week) covariates
3. Generate quantile forecasts (P10-P90) with covariate adjustments
4. Select cost-optimal quantile: Cu/(Cu+Co_period) = 1.0/(1.0+0.6) = 0.625
5. Convert to order quantities using base-stock policy
6. Generate submission CSV

**Note**: This is a DEMO of quantile+covariate forecasting capabilities, not competing with the official hierarchical Bayes submission.


## 1. Setup


In [1]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')

# TimesFM
import torch
timesfm_path = Path("..").resolve()
if str(timesfm_path) not in sys.path:
    sys.path.insert(0, str(timesfm_path))
import timesfm

# VN2 policy helpers
vn2_path = Path("../../vn2inventory").resolve()
if str(vn2_path) not in sys.path:
    sys.path.insert(0, str(vn2_path))
from vn2inventory.policy import compute_orders

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")


PyTorch: 2.8.0
CUDA available: False


In [2]:
# VN2 Configuration
DATA_DIR = Path("../../vn2inventory/data").resolve()
SUB_DIR = Path("../../vn2inventory/submissions").resolve()
SUB_DIR.mkdir(exist_ok=True)

# Costs (from VN2 competition)
SHORTAGE_COST = 1.0
HOLDING_COST = 0.2  # per unit per week
LEAD_WEEKS = 2
REVIEW_WEEKS = 1
PROTECTION_WEEKS = LEAD_WEEKS + REVIEW_WEEKS  # 3 weeks

# Critical fractile (Newsvendor optimal service level)
# Must use costs on the same time basis (protection period)
Co_period = HOLDING_COST * PROTECTION_WEEKS  # Convert weekly → protection-period
CRITICAL_RATIO = SHORTAGE_COST / (SHORTAGE_COST + Co_period)

print(f"\\n📊 VN2 Cost Structure:")
print(f"  Shortage cost (Cu): ${SHORTAGE_COST} per unit")
print(f"  Holding cost (Co): ${HOLDING_COST} per unit per week")
print(f"  Protection period (L+R): {PROTECTION_WEEKS} weeks")
print(f"  Holding cost (period basis): ${Co_period:.2f}")
print(f"  Critical fractile τ: {CRITICAL_RATIO:.4f} ({CRITICAL_RATIO*100:.2f}%)")


\n📊 VN2 Cost Structure:
  Shortage cost (Cu): $1.0 per unit
  Holding cost (Co): $0.2 per unit per week
  Protection period (L+R): 3 weeks
  Holding cost (period basis): $0.60
  Critical fractile τ: 0.6250 (62.50%)


## 2. Load VN2 Data


In [ ]:
# Load sales history and master data
sales_df = pd.read_csv(DATA_DIR / "Week 0 - 2024-04-08 - Sales.csv")
master_df = pd.read_csv(DATA_DIR / "Week 0 - Master.csv")
initial_state = pd.read_csv(DATA_DIR / "Week 0 - 2024-04-08 - Initial State.csv")
template = pd.read_csv(DATA_DIR / "Week 0 - Submission Template.csv")

# Strip whitespace from column names (avoid Store_x/Store_y conflicts)
sales_df.columns = sales_df.columns.str.strip()
master_df.columns = master_df.columns.str.strip()

# Convert to long format
id_cols = ["Store", "Product"]
assert set(id_cols).issubset(sales_df.columns), f"Missing id columns in sales_df"

sales_long = sales_df.melt(id_vars=id_cols, var_name="date", value_name="sales_qty")

# Normalize columns and types
sales_long.columns = sales_long.columns.str.strip()
sales_long["date"] = pd.to_datetime(sales_long["date"], errors="coerce")
sales_long["sales_qty"] = pd.to_numeric(sales_long["sales_qty"], errors="coerce").fillna(0.0)
sales_long["Store"] = pd.to_numeric(sales_long["Store"], errors="coerce").fillna(0).astype(int)
sales_long["Product"] = pd.to_numeric(sales_long["Product"], errors="coerce").fillna(0).astype(int)

# Add temporal features for dynamic covariates
sales_long["week_of_year"] = sales_long["date"].dt.isocalendar().week.astype(int)
sales_long["month"] = sales_long["date"].dt.month.astype(int)
sales_long["quarter"] = sales_long["date"].dt.quarter.astype(int)

# Merge master data (only ProductGroup and Department to avoid Store conflict)
master_cols = ["Product", "ProductGroup", "Department"]
master_subset = master_df[master_cols].drop_duplicates(subset=["Product"])
sales_long = sales_long.merge(master_subset, on="Product", how="left")

# Fill missing master data
sales_long["ProductGroup"] = sales_long["ProductGroup"].fillna(0).astype(int)
sales_long["Department"] = sales_long["Department"].fillna(0).astype(int)

# Sort
sales_long = sales_long.sort_values(["Store", "Product", "date"]).reset_index(drop=True)

# Get SKU list (all 599)
sku_list = template[["Store", "Product"]].copy()

print(f"\\n📦 Data Loaded:")
print(f"  Total SKUs: {len(sku_list)}")
print(f"  Sales history: {sales_long.shape}")
print(f"  Columns: {list(sales_long.columns)}")
print(f"  Weeks of data: {sales_long['date'].nunique()}")
print(f"  Date range: {sales_long['date'].min()} to {sales_long['date'].max()}")
print(f"✓ Master data merged (ProductGroup, Department)")
print(f"✓ Temporal features added (week_of_year, month, quarter)")


\n📦 Data Loaded:
  Total SKUs: 599
  Sales history: (94043, 4)
  Weeks of data: 157
  Date range: 2021-04-12 00:00:00 to 2024-04-08 00:00:00


## 3. Load TimesFM 2.5 with Quantile Head


In [4]:
print("Loading TimesFM 2.5...")
model = timesfm.TimesFM_2p5_200M_torch.from_pretrained("google/timesfm-2.5-200m-pytorch")

print("Compiling with quantile head...")
model.compile(
    timesfm.ForecastConfig(
        max_context=512,
        max_horizon=128,
        normalize_inputs=True,
        use_continuous_quantile_head=True,
        force_flip_invariance=True,
        infer_is_positive=True,
        fix_quantile_crossing=True,
    )
)
print("✓ Model ready with quantile forecasting enabled")


Loading TimesFM 2.5...
Compiling with quantile head...
✓ Model ready with quantile forecasting enabled


## 3.5. Prepare Covariates

Extract static and dynamic covariates for all SKUs to enhance forecasts.


In [ ]:
# Prepare covariates for all SKUs
CONTEXT_LENGTH = 140
HORIZON = 3

# Verify required columns exist
required_cols = {"Store", "Product", "date", "sales_qty", "month", "week_of_year", "quarter", "ProductGroup", "Department"}
missing = required_cols - set(sales_long.columns)
assert not missing, f"Missing required columns in sales_long: {missing}"

static_categorical_covariates = {"store": [], "product_group": [], "department": []}
static_numerical_covariates = {"mean_demand": [], "cv_demand": []}
dynamic_categorical_covariates = {"month": [], "week_of_year": [], "quarter": []}
dynamic_numerical_covariates = {"week_index": []}

for idx, row in sku_list.iterrows():
    sku_data = sales_long[
        (sales_long["Store"] == row["Store"]) &
        (sales_long["Product"] == row["Product"])
    ].sort_values("date")
    
    if sku_data.empty:
        continue
    
    # Static features
    static_categorical_covariates["store"].append(int(row["Store"]))
    static_categorical_covariates["product_group"].append(int(sku_data.iloc[0]["ProductGroup"]))
    static_categorical_covariates["department"].append(int(sku_data.iloc[0]["Department"]))
    
    mean_demand = float(sku_data["sales_qty"].mean())
    std_demand = float(sku_data["sales_qty"].std())
    cv_demand = (std_demand / mean_demand) if mean_demand > 0 else 0.0
    static_numerical_covariates["mean_demand"].append(mean_demand)
    static_numerical_covariates["cv_demand"].append(cv_demand)
    
    # Dynamic features (context + horizon)
    # We need CONTEXT_LENGTH + HORIZON values for dynamic covariates
    history = sku_data if len(sku_data) >= CONTEXT_LENGTH else sku_data
    context_data = history.iloc[-CONTEXT_LENGTH:] if len(history) >= CONTEXT_LENGTH else history
    
    # For horizon, we need future covariates
    # Since we don't have actual future dates, extrapolate from last known date
    last_date = context_data["date"].iloc[-1]
    future_dates = pd.date_range(start=last_date + pd.Timedelta(weeks=1), periods=HORIZON, freq="W-MON")
    
    # Combine context + horizon temporal features
    context_months = context_data["month"].tolist()
    context_weeks = context_data["week_of_year"].tolist()
    context_quarters = context_data["quarter"].tolist()
    
    future_months = [int(d.month) for d in future_dates]
    future_weeks = [int(d.isocalendar().week) for d in future_dates]
    future_quarters = [int((d.month - 1) // 3 + 1) for d in future_dates]
    
    dynamic_categorical_covariates["month"].append(context_months + future_months)
    dynamic_categorical_covariates["week_of_year"].append(context_weeks + future_weeks)
    dynamic_categorical_covariates["quarter"].append(context_quarters + future_quarters)
    dynamic_numerical_covariates["week_index"].append(list(range(len(context_data) + HORIZON)))

print(f"\\n✓ Covariates prepared for {len(sku_list)} SKUs")
print(f"  Static categorical: {list(static_categorical_covariates.keys())}")
print(f"  Static numerical: {list(static_numerical_covariates.keys())}")
print(f"  Dynamic categorical: {list(dynamic_categorical_covariates.keys())}")
print(f"  Dynamic numerical: {list(dynamic_numerical_covariates.keys())}")


## 4. Generate Quantile Forecasts (All SKUs)


In [ ]:
# Prepare inputs for all SKUs
inputs = []
for idx, row in sku_list.iterrows():
    sku_data = sales_long[
        (sales_long["Store"] == row["Store"]) &
        (sales_long["Product"] == row["Product"])
    ].sort_values("date")
    
    history = sku_data["sales_qty"].values
    inputs.append(history[-CONTEXT_LENGTH:] if len(history) >= CONTEXT_LENGTH else history)

print(f"\\n🔮 Generating quantile forecasts WITH COVARIATES for {len(inputs)} SKUs...")
print(f"  Context: {CONTEXT_LENGTH} weeks")
print(f"  Horizon: {HORIZON} weeks")
print(f"  Covariates: Store, ProductGroup, Department, temporal features")

# Generate quantile forecasts WITH COVARIATES
combined_forecast, xreg_forecast, quantile_forecast_list = model.forecast_with_covariates(
    horizon=HORIZON,
    inputs=inputs,
    static_categorical_covariates=static_categorical_covariates,
    static_numerical_covariates=static_numerical_covariates,
    dynamic_categorical_covariates=dynamic_categorical_covariates,
    dynamic_numerical_covariates=dynamic_numerical_covariates,
    xreg_mode="xreg + timesfm",  # Recommended mode
)

# Convert to arrays
point_forecast = np.array(combined_forecast)
quantile_forecast = np.array(quantile_forecast_list)

print(f"\\n✓ Forecasts generated with covariates!")
print(f"  Point forecast shape: {point_forecast.shape}")
print(f"  Quantile forecast shape: {quantile_forecast.shape}")
print(f"  Quantiles: [mean, P10, P20, P30, P40, P50, P60, P70, P80, P90]")

# Runtime assertion for API consistency
assert quantile_forecast.shape[-1] == 10, "Expected last dim = 10 ([mean, P10..P90])"
print("  ✓ Quantile tensor format validated")
print("  ✓ Covariate adjustments applied to all quantile levels")


\n🔮 Generating forecasts for 599 SKUs...
  Context: 140 weeks
  Horizon: 3 weeks
\n✓ Forecasts generated!
  Point forecast shape: (599, 3)
  Quantile forecast shape: (599, 3, 10)
  Quantiles: [mean, P10, P20, P30, P40, P50, P60, P70, P80, P90]
  ✓ Quantile tensor format validated


## 5. Select Cost-Optimal Quantile

Compute critical fractile τ = Cu/(Cu+Co_period), then round to nearest available quantile in grid [P10..P90].


In [6]:
# Map critical fractile to closest available quantile in grid
quantile_levels = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
closest_idx = np.argmin(np.abs(quantile_levels - CRITICAL_RATIO))
chosen_quantile = quantile_levels[closest_idx]

print(f"\\n💰 Cost-Optimal Quantile Selection:")
print(f"  Critical fractile τ: {CRITICAL_RATIO:.4f} (≈ P{int(CRITICAL_RATIO*100)})")
print(f"  Closest grid quantile: P{int(chosen_quantile*100)} (nearest to τ*={CRITICAL_RATIO:.3f})")
print(f"  Protection period: {PROTECTION_WEEKS} weeks (lead + review)")
print(f"  This means: Order to satisfy demand in {chosen_quantile*100:.0f}% of protection-period scenarios")

# Extract chosen quantile (add 1 for mean offset in array)
optimal_quantile_forecast = quantile_forecast[:, :, closest_idx + 1]
print(f"\\n  Selected forecast shape: {optimal_quantile_forecast.shape}")


\n💰 Cost-Optimal Quantile Selection:
  Critical fractile τ: 0.6250 (≈ P62)
  Closest grid quantile: P60 (nearest to τ*=0.625)
  Protection period: 3 weeks (lead + review)
  This means: Order to satisfy demand in 60% of protection-period scenarios
\n  Selected forecast shape: (599, 3)


## 6. Aggregate to Protection Period

Sum 3-week forecasts and estimate uncertainty


In [7]:
# Build demand_stats for policy
demand_stats = []

for i, (idx, sku_row) in enumerate(sku_list.iterrows()):
    # 3-week total demand (sum across horizon)
    demand_3w = optimal_quantile_forecast[i, :].sum()
    
    # Estimate std from P80-P50 spread (Normal approx)
    # Note: Summing weekly quantiles is an approximation;
    # it slightly underestimates tail risk if weeks are positively correlated.
    # For high-stakes items, simulate the period distribution instead.
    q50 = quantile_forecast[i, :, 5].sum()  # Median over 3 weeks
    q80 = quantile_forecast[i, :, 8].sum()  # P80 over 3 weeks
    
    # Rough std estimate: sigma ≈ (P80-P50) / 0.8416  (z_0.80 ≈ 0.8416)
    # Alternative: sigma ≈ (P90 - P10) / 2.5632 (more robust)
    std_3w = max((q80 - q50) / 0.8416, demand_3w * 0.1)  # Floor at 10% of mean
    
    demand_stats.append({
        "Store": sku_row["Store"],
        "Product": sku_row["Product"],
        "mean_demand": float(demand_3w),
        "std_demand": float(std_3w)
    })

demand_stats_df = pd.DataFrame(demand_stats).set_index(["Store", "Product"])

print(f"\\n📊 Demand Statistics (3-week protection period):")
print(f"  Mean demand: {demand_stats_df['mean_demand'].mean():.2f} units")
print(f"  Mean std: {demand_stats_df['std_demand'].mean():.2f} units")
print(f"  Max demand: {demand_stats_df['mean_demand'].max():.0f} units")
demand_stats_df.head()


\n📊 Demand Statistics (3-week protection period):
  Mean demand: 9.13 units
  Mean std: 6.50 units
  Max demand: 302 units


mean_demand  std_demand
Store Product                         
0     126         7.087064    7.419773
      182         3.162152    2.744097
1     124        24.379612   13.054598
2     124        26.680462   13.731553
      126         7.508129    7.485362

## 7. Generate Orders Using Base-Stock Policy


In [8]:
# Prepare current state
state = initial_state[["Store", "Product", "End Inventory", "In Transit W+1", "In Transit W+2"]].copy()
state.rename(columns={"End Inventory": "on_hand"}, inplace=True)
state["on_order"] = state[["In Transit W+1", "In Transit W+2"]].sum(axis=1)
current_state = state[["Store", "Product", "on_hand", "on_order"]].set_index(["Store", "Product"])

# Get index from template
index_df = template[["Store", "Product"]].set_index(["Store", "Product"])

# Compute orders
print("\\n🎯 Computing orders...")
orders = compute_orders(
    index_df=index_df,
    demand_stats=demand_stats_df,
    current_state=current_state,
    lead_time_weeks=LEAD_WEEKS,
    review_period_weeks=REVIEW_WEEKS,
    shortage_cost_per_unit=SHORTAGE_COST,
    holding_cost_per_unit_per_week=HOLDING_COST,
)

print(f"\\n✓ Orders computed for {len(orders)} SKUs")
print(f"  Total units: {orders.sum():,.0f}")
print(f"  Mean order: {orders.mean():.2f} units")
print(f"  Median order: {orders.median():.0f} units")
print(f"  Max order: {orders.max():.0f} units")
print(f"  Zero orders: {(orders == 0).sum()} SKUs")


\n🎯 Computing orders...
\n✓ Orders computed for 599 SKUs
  Total units: 17,454
  Mean order: 29.14 units
  Median order: 12 units
  Max order: 924 units
  Zero orders: 3 SKUs


## 8. Save Submission


In [ ]:
# Create submission DataFrame
submission = index_df.copy()
submission["0"] = orders.values

# Validate
assert len(submission) == len(template), "Row count mismatch!"
assert submission.index.equals(template.set_index(["Store", "Product"]).index), "Index mismatch!"

# Save
output_path = SUB_DIR / "orders_timesfm_quantile_covariates_demo.csv"
submission.to_csv(output_path)

print(f"\\n✅ SUBMISSION SAVED: {output_path}")
print(f"\\n📄 Submission Summary:")
print(f"  Rows: {len(submission)}")
print(f"  Columns: {list(submission.columns)}")
print(f"  Total units ordered: {submission['0'].sum():,.0f}")
print(f"  Mean order: {submission['0'].mean():.2f}")
print(f"  Zeros: {(submission['0'] == 0).sum()}")

print(f"\\n🔝 Top 10 Orders:")
top_10 = submission.nlargest(10, "0")
print(top_10)

print(f"\\n💡 This demo shows TimesFM 2.5's quantile + covariate forecasting capability.")
print(f"   Covariates (Store, ProductGroup, Department, temporal) enhance distributional forecasts.")
print(f"   Cost-optimal quantile (P60) was automatically selected based on Cu/Co ratio.")
print(f"   Ready for VN2 submission validation!")


\n✅ SUBMISSION SAVED: /Users/senoni/noni/vn2inventory/submissions/orders_timesfm_quantile_demo.csv
\n📄 Submission Summary:
  Rows: 599
  Columns: ['0']
  Total units ordered: 17,454
  Mean order: 29.14
  Zeros: 3
\n🔝 Top 10 Orders:
                 0
Store Product     
61    23       924
      124      703
63    124      596
60    125      582
62    23       557
60    23       489
63    23       485
64    17       396
      23       351
61    48       289
\n💡 This demo shows TimesFM 2.5's quantile forecasting capability.
   Cost-optimal quantile (P80) was automatically selected based on Cu/Co ratio.
   Ready for VN2 submission validation!
